In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
olympic_podium=pd.read_csv('../data/clean/dataset_olympics_clean/podium_1996-2016.csv')
olympic_podium.region.fillna("Singapore",inplace=True)
olympic_podium.region.isnull().sum()
olympic_podium=olympic_podium[olympic_podium.region !="Kosovo"]
olympic_podium

In [ ]:
time_zones=pd.read_csv("../data/clean/dataset_olympics_clean/time_zones2.csv", encoding="latin-1", sep=";")
time_zones

In [ ]:
time_zones_athlete=time_zones

In [ ]:
time_zones_athlete[["Hours","Minutes"]]=time_zones_athlete["GMT offset"].str.split(":", expand=True)
time_zones_athlete
time_zones_athlete.Minutes.fillna("00",inplace=True)
time_zones_athlete.head(40)

time_zones_athlete["Minutes"]=time_zones_athlete["Minutes"].map({
    "30": 0.5,
    "00": 0,
    "0": 0
})
time_zones_athlete["Hours"]=time_zones_athlete["Hours"].astype(float).abs()
time_zones_athlete["GMT Athlete Country"]=time_zones_athlete["Hours"]+time_zones_athlete["Minutes"]

time_zones_athlete

In [ ]:
time_zones_athlete.rename(columns={"Country":'region'}, inplace=True)
time_zones_athlete

In [ ]:
olympic_podium=pd.merge(
    olympic_podium, 
    time_zones_athlete[["region","GMT Athlete Country","Capital city"]],
    on="region",
    how="left")

olympic_podium

In [ ]:
print(olympic_podium.region.isnull().sum())
print(olympic_podium["GMT Athlete Country"].isnull().sum())
print(olympic_podium["Capital city"].isnull().sum())

In [ ]:
olympic_podium[olympic_podium["GMT Athlete Country"].isnull()].head()

In [ ]:
olympic_podium.dropna(subset=["GMT Athlete Country"], inplace=True)
print(olympic_podium.region.isnull().sum())
print(olympic_podium["GMT Athlete Country"].isnull().sum())
print(olympic_podium["Capital city"].isnull().sum())

In [ ]:
time_zones_host=time_zones_athlete
time_zones_host.rename(columns={"Capital city":'City'}, inplace=True)
time_zones_host.rename(columns={"GMT Athlete Country":'GMT Host City'}, inplace=True)
time_zones_host

In [ ]:
olympic_podium=pd.merge(
    olympic_podium, 
    time_zones_host[["City","GMT Host City"]],
    on="City",
    how="left")

olympic_podium

In [ ]:
olympic_podium.isnull().any()

In [ ]:
olympic_podium.info()

In [ ]:
olympic_podium["Time difference"]=(olympic_podium["GMT Host City"]-olympic_podium["GMT Athlete Country"]).abs()
olympic_podium

In [ ]:
olympic_podium.groupby("Time difference")["Medal"].count()


In [ ]:
def create_range_time(time):
    if time <= 2.0:
        return "[0-2.0]"
    elif time <=4:
        return "(2.0-4.0]"
    elif time <=6:
        return "(4.0-6.0]"
    elif time <=8:
        return "(6.0-8.0]"
    elif time <=10:
        return "(8.0-10.0]"
    elif time <=12:
        return "(10.0-12.0]"
    else:
        return "(12.0-13.00]"

In [ ]:
olympic_podium["Range Time"]=olympic_podium["Time difference"].apply(create_range_time)
olympic_podium

In [ ]:
olympic_podium.groupby("Range Time")["Medal"].count().sort_values(ascending=False)

In [ ]:
olympic_podium_range_df=pd.DataFrame(olympic_podium.groupby("Range Time")["Medal"].count().sort_values(ascending=False)).reset_index()
olympic_podium_range_df

In [ ]:
import plotly.express as px
fig1 = px.bar(olympic_podium_range_df, x = 'Range Time', y = 'Medal')

fig1.show()

In [ ]:
olympic_podium_corr=olympic_podium
olympic_podium_corr

In [ ]:
olympic_podium_corr["Time difference"]=olympic_podium_corr["Time difference"].map({
    0.0: 0.0,
    0.5: 1.0,
    1.5: 2.0,
    2.5: 3.0,
    3.5: 4.0,
    4.5: 5.0,
    5.5: 6.0,
    6.5: 4.0,
    7.5: 8.0,
    8.5: 9.0,
    9.5: 10.0,
    10.5: 11.0,
    11.5: 12.0,
    12.5: 13.0,
    1.0: 1.0,
    2.0: 2.0,
    3.0: 3.0,
    4.0: 4.0,
    5.0: 5.0,
    6.0: 6.0,
    7.0: 7.0,
    8.0: 8.0,
    9.0: 9.0,
    10.0: 10.0,
    11.0: 11.0,
    12.0: 12.0,
    13.0: 13.0,
    
})

olympic_podium_corr

In [ ]:
olympic_podium_corr["Time difference"].unique()

In [ ]:
olympic_podium_corr.groupby("Time difference")["Medal"].count()

In [ ]:
olympic_podium_corr_df=pd.DataFrame(olympic_podium_corr.groupby("Time difference")["Medal"].count()).reset_index()
olympic_podium_corr_df

In [ ]:
print('Correlación Pearson: ', olympic_podium_corr_df['Time difference'].corr(olympic_podium_corr_df['Medal'], method='pearson'))

In [ ]:
sns.regplot(x = olympic_podium_corr_df["Time difference"], y = olympic_podium_corr_df["Medal"])